# Recuperación de pacientes de XiO
-------------------
## Descripción
Mediante este cuaderno datos de *pacientes archivados en XiO se convierten a formato DICOM* para su importación en ARIA.

Los datos archivados en XiO están almacenados en un servidor del Hospital al que se accede mediante `ftp`

La conversión utiliza los programas `plastimatch`y `Velocity`

`plastimatch` recupera de los datos de XiO las imágenes de CT, las estructuras y la dosis.

`Velocity` añade el plan contenedor para poder importar la dosis en ARIA.

### Código

Importación de módulos

In [ ]:
# - Manipulación de archivos DICOM
import pydicom as dicom
# - Conversión del formato nativo de XiO a DICOM
import pyplastimatch as plast
# - Rutas y comprobaciones de existencia de ficheros
from os import path, getcwd
from pathlib import Path
from glob import glob
from sys import path as syspath

Definir el directorio de trabajo base.

Crear la carpeta `scripts` en el directorio de trabajo base y descargar los scripts que complementan el funcionamiento de `plastimatch`

In [ ]:
# Referenciar
baseworkdir = '/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO'
if not Path(baseworkdir).exists():
    !mkdir -p $baseworkdir

# Descargar los scripts de corrección
if Path(baseworkdir + '/scripts').exists():
  !rm -rf $baseworkdir/scripts
!mkdir -p $baseworkdir/scripts
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/xioss
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/genindex
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/deltact
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/dcmcoher.py

# Añadir al sistema la ruta de los scripts
if baseworkdir + '/scripts' not in syspath:
  syspath.insert(0, baseworkdir + '/scripts')

Definir el nombre del archivo que contiene los datos archivados en XiO 

In [ ]:
ArchivedPatient = '584953_SALDA_A_GARCES_SANTIAGO_Mar_14_2011.tgz'

Desarchivar los datos de XiO

In [ ]:
%%capture
!tar -xvzf $ArchivedPatient -C $baseworkdir

Mostrar la estructura de carpetas de los datos desarchivados

In [ ]:
!tree -d $baseworkdir/patient

Recuperar datos relevantes.

Número de historia y nombre y apellidos del paciente

In [ ]:
# Número de historia
PatientID = ArchivedPatient.split('_')[0]

# Nombre y apellidos
demographicfile = !find $baseworkdir/patient/$PatientID -name demographic -print
with open(demographicfile[0], 'r', encoding='latin') as demogf:
  demographicdata = demogf.read()

ApellidosNombre = demographicdata.split('\n')[2]

print(f'Historia: {PatientID}\nApellidos y nombre: {ApellidosNombre}')

Definir el estudio CT y el plan que se quiere recuperar

In [ ]:
CT = 'CT1'
Plan ='1FASE1B'

Recuperación de las estructuras

In [ ]:
# Si no existe crear el índice de imágenes CT y las máscaras `.mha` de todas las estructuras
if not Path(baseworkdir + '/patient/' + PatientID + '/anatomy/studyset/' + CT + '/index.dat').exists():
    # Copiar el script para la creación del índice de cortes de CT
    !cp $baseworkdir/scripts/genindex $baseworkdir/patient/$PatientID/anatomy/studyset/$CT
    
    # Adjudicar permisos de ejecución y crear el índice
    !chmod +x $baseworkdir/patient/$PatientID/anatomy/studyset/$CT/genindex
    cwd = getcwd()
    %cd -q $baseworkdir/patient/$PatientID/anatomy/studyset/$CT
    print('Generar el índice de imáagenes de CT...')
    !./genindex
    %cd -q $cwd

# Crear la carpeta para los archivos máscara con las estructuras
!mkdir -p $baseworkdir/patient/$PatientID/anatomy/studyset/$CT/ss

# Copiar el script para la creación de las estructuras
!cp $baseworkdir/scripts/xioss $baseworkdir/patient/$PatientID/anatomy/studyset/$CT

# Correr el script de recuperación de las estructuras
!chmod +x $baseworkdir/patient/$PatientID/anatomy/studyset/$CT/xioss
print('Recuperar estructuras...')
!$baseworkdir/patient/$PatientID/anatomy/studyset/$CT/xioss --input $baseworkdir/patient/$PatientID --plan $Plan --studyset $CT > xioss.log

Conversion del CT, las estructuas y la dosis desde formato XIO a formato DICOM 

In [ ]:
# Referencia al directorio de trabajo actual
cwd = getcwd()
# Cambiar al directorio base
%cd -q $baseworkdir

# Variables de pyplastimatch
verbose = True
path_to_logfile_xiodcm = './' + PatientID + '_xiodcm.log'

# Opciones de plastimatch convert
convert_args_rt ={'input' :  'patient/' + PatientID  + '/anatomy/studyset/' + CT + '/DCMData',
                  'input-prefix' : 'patient/' + PatientID  + '/anatomy/studyset/' + CT + '/ss',
                  'input-dose-xio' : 'patient/' + PatientID  + '/plan/' + Plan + '/dose.1',
                  'output-dicom' : 'xiodcm'}

# Correr el script de conversión de los datos XiO en estudio DICOM
plast.convert(verbose=verbose, path_to_log_file=path_to_logfile_xiodcm, **convert_args_rt )

# Volver al directorio inicial
%cd -q $cwd

Corrección de los marcos de referencia en los archivos DICOM

In [ ]:
# Referencia al directorio de trabajo actual
cwd = getcwd()
# Cambiar al directorio patient
%cd -q $baseworkdir/patient

# Copiar el script para determinar el delta entre los marcos de referencia del estudio CT y el plan
!cp $baseworkdir/scripts/deltact .

## Asegurar que el archivo se actualiza
if Path('./deltaframes').exists():
  !rm -f ./deltaframes
## Adjudicar permisos de ejecución y calcular el delta
!chmod +x deltact
!./deltact --input $PatientID --studyset $CT > deltact.log

## Regresar al directorio base
%cd -q $baseworkdir

# Importar las herramientas de corrección
from dcmcoher import correctImagePositionPatientInCTImages

# Establecer el mismo marco de referencia en todo el estudio
print('Establecer el mismo marco de referencia para las imágenes CT, las estructuras y la dosis...')
correctImagePositionPatientInCTImages(patientID=PatientID, studyset=CT)

# Volver al directorio inicial
%cd -q $cwd